In [ ]:
%pylab notebook

# Evaluate the results of anisotropic training

## Fetch the CMP data to compare against

In [ ]:
import os
import sys
import numpy as np
from glob import glob

eval_txt = 'data/training/independant_12_layers/fold_01/eval.txt'
eval_files = [fn.strip() for fn in open(eval_txt).readlines()]
print "Found {} eval files".format(len(eval_files))

cmp_eval_files = [f for f in eval_files if '/cmp/' in f]
print "Found", len(cmp_eval_files), "from the CMP dataset", len(eval_files)-len(cmp_eval_files), "are our own"

In [ ]:
from pyfacades.util.metrics import Metrics

In [ ]:
!nvidia-smi

In [ ]:
GPU=False
import caffe
if not GPU:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0)

In [ ]:
import skimage.io
from skimage.transform import warp, ProjectiveTransform

In [ ]:
from pyfacades.rectify import Homography as AffaraRectifier

In [ ]:
from pyfacades.util import Metrics, softmax, channels_first, channels_last

In [ ]:
from pyfacades.models.driving_12x360x480 import process_strip as segment_driving

In [ ]:
import pyfacades.models.disjoint_5_labels as segnet

In [ ]:
with warnings.catch_warnings('ignore'):
    from pyfacades.models.disjoint_5_labels.segment import process_strip
    import pyfacades.models.independant_12_layers as i12

## Put it all together 

In [ ]:
%pdb on

In [ ]:
print segnet.segment.LABELS
print i12.LABELS

In [ ]:
def get_metrics(f, vis=True, rectify=True):
    blob = np.load(f)
    rgb = blob[:3]  
    
    expected = blob[3:]  
    expected_windows = expected[i12.WINDOW]
 
    probs, confs = process_strip(rgb)
    predicted_windows = (probs[segnet.model.WINDOW]>0.5)*2
    
    expected_windows[expected[i12.BACKGROUND]==2] =1 # ignore
    
    # Remove padding from considereation
    padding = rgb.max(axis=0).max(axis=0) == 0
    padding = np.outer(np.ones(rgb.shape[1]), padding) > 0
    expected_windows[padding] = 1
    
    
    mf = Metrics(expected=expected_windows, 
             predicted=predicted_windows, 
             label_positive=2,
             label_negative=0,
             source=f,
             feature='window'
            )
    if vis:
        clf()
        subplot(221)
        imshow(channels_last(rgb)/255.)
        axis('off')
        subplot(223)
        imshow(expected_windows, vmin=0, vmax=2)
        axis('off')
        subplot(222)
        imshow(predicted_windows, vmin=0, vmax=2)
        axis('off')
        subplot(224)
        imshow(probs[segnet.model.WINDOW], vmin=0, vmax=1, cmap=cm.gray)
        axis('off')

        
        try:
            suptitle('A:{}, P:{}, R:{}'.format(mf.pixel_accuracy, mf.pixel_precision, mf.pixel_recall))
        except ZeroDivisionError:
            pass

    return mf, rgb, expected_windows, predicted_windows

figure(figsize=(8,8))
mf, rgb, expected, predicted = get_metrics(eval_files[12], rectify=True)

In [ ]:
import anydbm
import json
import hashlib
import munch

In [ ]:
eval_results = anydbm.open('eval_segnet_based_on_CMP', 'c')
checksum = hashlib.md5(open(segnet.model.WEIGHTS).read()).hexdigest()

In [ ]:
if 'md5' in eval_results and eval_results['md5'] == checksum:
    print "We already seem to have run evaluation..."

In [ ]:

recompute = False
visualize = False

if visualize:
    fig = figure(figsize=(6,6))


total = Metrics(feature='windows')
for i, f in enumerate(eval_files):   
    if recompute or f not in eval_results:
        mf, rgb, expected, predicted = get_metrics(f, visualize)
        eval_results[f] = json.dumps(mf.as_dict())
        if visualize:
            try:
                suptitle('{} of {}, $P$:{:.2%}, $R$:{:.2%}, $F_1$:{:.2%}, $A$:{:.2%}'.format(i, len(eval_files), total.pixel_precision, total.pixel_recall, total.pixel_f1, total.pixel_accuracy))
            except ZeroDivisionError:
                suptitle("Not enough samples yet....")         
            fig.canvas.draw()
    else:
        mf = Metrics(**json.loads(eval_results[f]))
    print '\r {} of {}'.format(i, len(eval_files)),
    total += mf
print '\rTotal:', total, "Acc:", total.pixel_accuracy

In [ ]:
cmp_total = Metrics()
for f in eval_results:
    if '/cmp/' in f:
        cmp_total += Metrics(**json.loads(eval_results[f]))
print "CMP:", cmp_total, 'Acc:', cmp_total.pixel_accuracy

In [ ]:
gsv_total = Metrics()
for f in eval_results:
    if '/cmp/' not in f:
        gsv_total += Metrics(**json.loads(eval_results[f]))
print "GSV:", gsv_total, 'Acc:', gsv_total.pixel_accuracy

# Why is recall so bad?

In [ ]:
#%%file color_coded_errors.py

ERROR_COLORS = [[128, 128,  128], #TN
                [255, 0,    0],   #FP
                [0,   255,  0],   #TP
                [0,   0,    255], #FN
                [0,   0,    0], #ignored
               ]  
ERROR_COLORS = array(ERROR_COLORS, dtype=np.uint8)

def color_coded_errors(expected, predicted, ignored=None, colors=ERROR_COLORS):
    TP = ~ignored & (expected & predicted)
    FN = ~ignored & (expected & ~predicted)
    FP = ~ignored & (~expected & predicted)
    TN = ~ignored & (~expected & ~predicted)
    errors = np.argmax(array([TN, FP, TP, FN, ignored], dtype=np.uint8), axis=0)
    if colors is not None:
        return np.ma.masked_array(colors[errors], np.dstack([TN]*3))
    else:
        return errors  
      
def render_errors(path, alpha=0.5, noFN=True):
    mf, rgb, expected, predicted = get_metrics(path)
    ignored = (expected != mf.label_positive) & (expected != mf.label_negative)
    expected = expected==mf.label_positive
    predicted = predicted==mf.label_positive
    print mf.label_positive
    cc = color_coded_errors(expected, predicted, ignored)
    rgb = rgb.transpose(1,2,0)/255.
    if noFN:
        rgb[~cc.mask] = (1-alpha)*rgb[~cc.mask] + alpha*cc[~cc.mask]
        return rgb.clip(0,1)
        #imshow(predicted)
    else:
        rgb = (1-alpha)*rgb + alpha*cc
    return rgb

In [ ]:
def score(f):
    acc = Metrics(**json.loads(eval_results[f])).pixel_f1
    if isnan(acc):
        acc = 0
    return acc

In [ ]:
accs = array([score(f) for f in eval_files])

cmp_accs = array([accs[i] for i in range(len(eval_files))  if '/cmp/' in eval_files[i]])
cmp_files = array([eval_files[i] for i in range(len(eval_files))  if '/cmp/' in eval_files[i] ])

gsv_accs = array([accs[i] for i in range(len(eval_files)) if '/cmp/' not in eval_files[i] ] )
gsv_files = array([eval_files[i] for i in range(len(eval_files))  if '/cmp/' not in eval_files[i] ])

gsv_ranking = argsort(gsv_accs)
cmp_ranking = argsort(cmp_accs)

In [ ]:
figure()
plot(gsv_accs[gsv_ranking])

In [ ]:
for i in range(9):
    cached = 'segnet-GSV-top-{}.png'.format(i+1)
    os.remove(cached)

In [ ]:
gsv_comparison_files = [fn.strip() for fn in open('gsv_comparison_files.txt')]

import os
fig = figure(figsize=(9,9))
plt.subplots_adjust(wspace=0)
for i in range(9):
    subplot(3,3,i+1)
    cached = 'segnet-GSV-top-{}.png'.format(i+1)
    if os.path.isfile(cached):
        err_image = imread(cached)
    else:
        err_image = render_errors(gsv_comparison_files[i], alpha=0.6);
        imsave(cached, err_image)
    imshow(err_image)
    xticks([]); yticks([]); 
    #xlabel(cached, fontsize=8)
    fig.canvas.draw()
fig.tight_layout()
savefig('segnet-GSV-top-9-figure.png', dpi=400)

In [ ]:
cmp_comparison_files = [fn.strip() for fn in open('cmp_comparison_files.txt')]

import os
fig = figure(figsize=(9,9))
plt.subplots_adjust(wspace=0)
for i in range(9):
    subplot(3,3,i+1)
    cached = 'segnet-CMP-top-{}.png'.format(i+1)
    #os.remove(cached)
    if os.path.isfile(cached):
        err_image = imread(cached)
    else:
        err_image = render_errors(cmp_comparison_files[i], alpha=0.6);
        imsave(cached, err_image)
    imshow(err_image)
    xticks([]); yticks([]);
    #xlabel(cached, fontsize=8)
    fig.canvas.draw()
fig.tight_layout()
savefig('segnet-CMP-top-9-figure.png', dpi=400)